In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.4.0+cu121
12.1


In [3]:
#Install required packages.
#!pip install progress progressbar2 alive-progress tqdm
#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
#!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
# !pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
#!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.3.1+cu121.html
#!pip install torch-geometric

In [4]:
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple progress progressbar2 alive-progress tqdm
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple torch-scatter -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple torch-sparse -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple torch-cluster -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple torch-geometric


In [2]:
# memory footprint support libraries/code
#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
#!pip install gputil
#!pip install psutil
#!pip install humanize

import psutil
import humanize
import os
#import GPU as GPU
#GPUs = GPU.getGPUs()
'''
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()
'''

'\n# XXX: only one GPU on Colab and isn’t guaranteed\ngpu = GPUs[0]\ndef printm():\n  process = psutil.Process(os.getpid())\n  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))\n  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))\n\nprintm()\n'

In [3]:
import numpy as np
np.random.seed(0)
import os, glob
import time
import h5py
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import ConcatDataset, Dataset, DataLoader, sampler, DistributedSampler
#from torch.utils.data import *
from sklearn.metrics import roc_curve, auc

In [4]:
import argparse
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=5, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=64, help='Initial learning rate.') #100
parser.add_argument('--maxnodes', type=int, default=1000, help='max nodes.') #100
parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.3, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [5]:
import torch
import torch.nn.functional
import torch_geometric
from torch_geometric.data import Data
class ParquetDataset(Dataset):
    def __init__(self, filename):
        self.parquet = pq.ParquetFile(filename)
        self.cols = None # read all columns
        #self.cols = ['X_jets.list.item.list.item.list.item','y']
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        #print(data.keys())
        data['X_jet'] = torch.tensor(np.float32(data['X_jet']))
        data['y'] = torch.tensor(np.float32(data['y']))
        #data['m0'] = torch.tensor(np.float32(data['m0']))
        #data['pt'] = torch.tensor(np.float32(data['pt']))
        data['X_jet'][data['X_jet'] < 1.e-3] = 0.
        # Preprocessing
        #data['nonzeroPixels'][data['nonzeroPixels'] < 1.e-3] = 0. # Zero-Suppression
        #data['nonzeroPixels'][-1,...] = 25.*data['nonzeroPixels'][-1,...] # For HCAL: to match pixel intensity distn of other layers
        #data['nonzeroPixels'] = data['nonzeroPixels']/100. # To standardize
        return dict(data)
    def __len__(self):
        return self.parquet.num_row_groups

In [6]:
# Boosted Top Jets
decays = sorted(glob.glob('/pscratch/sd/b/bbbam/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To3p6_m14p8To17p2_dataset_2_unbaised_v2_train/*m1p2To3p6*0000*'))
print("toal files  ", len(decays))               

dset_train = ConcatDataset([ParquetDataset(d) for d in decays])


toal files   1


In [7]:
## generate list to count nodes for each graph
def nodeCounter(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

def ref(bsize,nodeC,i1,i2):
  maxC=np.max(np.array(nodeC))
  maxC=args.maxnodes#maxC + (4 - maxC % 4) ##max num of nodes 1161%4
  refMat=np.zeros((bsize,maxC)) ## matrix of zeros
  for pi in range(i1,i2):##10
    refMat[pi,:nodeC[pi]]=1 ## fill ones
  return refMat,maxC

def assigner(nodelist):
  fin=[]
  countit=0
  for m in nodelist:
      fin.append(np.repeat(countit,m))
      countit+=1
  return np.array(fin)

In [11]:
#!pip install torch_geometric

In [8]:
# Boosted Top Jets without GPU
import torch_geometric.transforms
from torch_geometric.nn import knn_graph
import torch_geometric.data
import torch
from torch_geometric.data import Data
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import progressbar
from progressbar import Bar
from IPython.display import HTML, display

def progress(value, max=3200):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 75%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

def convert_to_graph(train_data, start_idx, end_idx, granularity=1, new_file=False):   # input data format should be [N,C,H,W]
    print('Processing idx nos. from '+str(start_idx)+' to '+str(end_idx))
    out = display(progress(start_idx, end_idx), display_id=True)
    for idx in range(start_idx,end_idx):

      data=train_data[idx]['X_jet']
      min_pixel_val = torch.min(data)
      max_pixel_val = torch.max(data)
      image3D = data.reshape(125*granularity,125*granularity,13)
      Hcal_frame = torch.zeros_like(image3D[:,:,4])
      for i in range(0,image3D.shape[0]):
        for j in range(0,image3D.shape[1]):
          if (i-2)%5 == 0:
            if (j-2)%5 == 0:
              Hcal_frame[i,j] = torch.sum(image3D[i-2:i+3,j-2:j+3,4])
            else:
              Hcal_frame[i,j] = 0.
          else:
            Hcal_frame[i,j] = 0.
      image3D[:,:,4] = Hcal_frame
      nonzero_pos = torch.nonzero(image3D, as_tuple=True)
      coords = torch.cat((torch.unsqueeze(nonzero_pos[0],dim=1), torch.unsqueeze(nonzero_pos[1],dim=1)),dim=1).float()
      coords = (coords.float() - 62)/62

      Ecal = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],3],dim=1)
      dz = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],2],dim=1)
      d0 = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],1],dim=1)
      Hcal = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],4],dim=1)
      pT = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],0],dim=1)
      BPIX1 = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],5],dim=1)
      BPIX2 = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],6],dim=1)
      BPIX3 = torch.unsqueeze(image3D[nonzero_pos[0],nonzero_pos[1],7],dim=1)
      feats = torch.cat((coords[:,0:1],coords[:,1:], pT, d0, dz, Ecal, Hcal, BPIX1, BPIX2, BPIX3), dim=1)
      feats = torch.unique(feats,dim=0)
      edge_index = knn_graph(feats[:,0:2], k=16, batch=None, loop=True)  ## Create knn graph adjacency matrix
      #print(coords.shape, edge_index.shape, pT.shape, Ecal.shape, Hcal.shape, dz.shape, d0.shape, BPIX1.shape, BPIX2.shape, BPIX3.shape)

      #parquet_Dataframe = pd.DataFrame({'coords0':[np.array(coords[:,0].cpu())], 'coords1':[np.array(coords[:,1].cpu())], 'edge_index_from': [np.array(edge_index[0,:].cpu())], 'edge_index_to': [np.array(edge_index[1,:].cpu())], 'pT': [np.array(torch.squeeze(pT).cpu())], 'ECAL':[np.array(torch.squeeze(Ecal).cpu())], 'HCAL':[np.array(torch.squeeze(Hcal).cpu())], 'd0':[np.array(torch.squeeze(d0).cpu())], 'dz':[np.array(torch.squeeze(dz).cpu())], 'BPIX1':[np.array(torch.squeeze(BPIX1).cpu())], 'BPIX2': [np.array(torch.squeeze(BPIX2).cpu())], 'BPIX3': [np.array(torch.squeeze(BPIX3).cpu())], 'y':np.array(train_data[idx]['y']), 'tfrecord': train_data[idx]['tfrecord'], 'm0': train_data[idx]['m0'], 'pT_jet': train_data[idx]['pt']})
      parquet_Dataframe = pd.DataFrame({'coords0':[np.array(feats[:,0].cpu())],'coords1':[np.array(feats[:,1].cpu())],'edge_index_from': [np.array(edge_index[0,:].cpu())],'edge_index_to': [np.array(edge_index[1,:].cpu())],'pT':[np.array(feats[:,2].cpu())],'d0':[np.array(feats[:,3].cpu())],'dz':[np.array(feats[:,4].cpu())],'ECAL':[np.array(feats[:,5].cpu())],'HCAL':[np.array(feats[:,6].cpu())],'BPIX1':[np.array(feats[:,7].cpu())],'BPIX2':[np.array(feats[:,8].cpu())],'BPIX3':[np.array(feats[:,9].cpu())],'y':np.array(train_data[idx]['y']), 'tfrecord': train_data[idx]['tfrecord'], 'm0': train_data[idx]['m0'], 'pT_jet': train_data[idx]['pt']})
      table = pa.Table.from_pandas(parquet_Dataframe)
      #print(type(table.schema), type(table))

      if (new_file):
      # create a parquet write object giving it an output file
        output_filename = 'data_shared/BoostedTopParquet_x1_fixed_Graphs/BoostedTop_x1_train_samples_'+str(start_idx)+'_to_'+str(end_idx-1)+'.parquet'
        pqwriter = pq.ParquetWriter(output_filename,table.schema,compression='snappy')
        new_file=False
      pqwriter.write_table(table)
      out.update(progress(idx, end_idx))
      if (idx%10==0):
        print("Current idx: ",str(idx))

    if pqwriter:
      pqwriter.close()

    return True

In [9]:
# Boosted Top Jets
convert_to_graph(dset_train,0,100,granularity=1,new_file=True)

Processing idx nos. from 0 to 100


ImportError: 'knn_graph' requires 'torch-cluster'

In [ ]:
# Boosted Top Jets
convert_to_graph(dset_train,6400*51,6400*52,granularity=1,new_file=True)

Processing idx nos. from 326400 to 332800


Current idx:  326400
Current idx:  329600


True

In [ ]:
dset_train[640000-1]

{'X_jets': tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 'm0': tensor([69.3547]),
 'pt': tensor([626.9458]),
 'tfrecord': ['data_shared/BoostedTop_x1_fixed_tfrecord/BoostedJets_fullSample_x1_file-143'],
 'y': tensor([0.])}

In [ ]:
tp = dset_test[75]['ECAL'].detach()

In [ ]:
dset_test[75]['ECAL'][-1]

tensor(0.)

In [ ]:
tp[-1]-=1

In [ ]:
tp

tensor([0.0212, 0.0212, 0.0521,  ..., 0.0000, 0.0000, 0.0000])